In [8]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score, GridSearchCV
import pandas as pd
import pickle 
import numpy as np
from tqdm import tqdm
from sklearn.feature_selection import RFE, SelectFromModel
from sklearn.metrics import roc_auc_score

In [9]:
def get_pickles(): 
    x_train = pickle.load(open(f'../../../data/processed/pickles/cluster_x_train.p', 'rb'))
    x_test = pickle.load(open(f'../../../data/processed/pickles/cluster_x_test.p', 'rb'))
    y_train = pickle.load(open(f'../../../data/processed/pickles/cluster_y_train.p', 'rb'))
    y_test = pickle.load(open(f'../../../data/processed/pickles/cluster_y_test.p', 'rb'))
    X = pickle.load(open('../../../data/processed/pickles/cluster_X.p', 'rb'))
    y = pickle.load(open('../../../data/processed/pickles/cluster_y.p', 'rb'))

    return (x_train, x_test, y_train, y_test), (X,y)

(x_train, x_test, y_train, y_test), (X,y) = get_pickles()


print(f'Original X: {X.shape}\tOriginal y: {y.shape}')
print(f'Train X: {x_train.shape}\tTrain y: {y_train.shape}')
print(f'Test X: {x_test.shape}\tTest y: {y_test.shape}')

y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

Original X: (57247, 55)	Original y: (57247, 1)
Train X: (42935, 54)	Train y: (42935, 1)
Test X: (14312, 54)	Test y: (14312, 1)


In [12]:
#Vanilla 
knn = KNeighborsClassifier() 
knn.fit(x_train,y_train)
test_pred = knn.predict(x_test)
print(f'Vanilla ROC Score: {roc_auc_score(y_test, test_pred)}')

Vanilla ROC Score: 0.7142552307146847


In [10]:
knn = KNeighborsClassifier()
best_score = 0 
best_i = 0
r_back = [i for i in range(1,len(x_train.columns)+1)]
pbar = tqdm(r_back)
for i in pbar: 
    pbar.set_description(f'Testing {i} PCA Features || Best AUC: {best_score} || Best n: {best_i}')
    pca = PCA(n_components = i)
    pca.fit(x_train)
    x_train_new = pca.transform(x_train)
    x_test_new = pca.transform(x_test)
    knn.fit(x_train_new,y_train)
    pred = knn.predict(x_test_new)
    roc_auc = roc_auc_score(y_test, pred)
    if roc_auc > best_score: 
        best_score = roc_auc 
        best_i = i
    #masks for columns that are important
        pickle.dump(best_i, open('../../../models/KNN_PCA.p', 'wb'))

Testing 54 PCA Features || Best AUC: 0.7103010768779537 || Best n: 31: 100%|██████████████████████████████████████████████████████████████████████████████| 54/54 [01:25<00:00,  1.59s/it]


In [ ]:

param_grid = {
   
    
}
x_train_new = x_train
x_test_new= x_test
lda = LinearDiscriminantAnalysis()
gs = GridSearchCV(lda, param_grid = param_grid, verbose = 2, n_jobs = 10)
gs.fit(x_train_new, y_train)

In [ ]:
pickle.dump(gs, open('../../models/KNN_GridSearch.p', 'wb'))
gs.best_estimator_

In [ ]:
knn_tuned = KNeighborsClassifier()

knn_tuned.fit(x_train_new,y_train)
test_pred = knn_tuned.predict(x_test_new)
print(f'Tuned ACC Score: {knn_tuned.score(x_test_new,y_test)}')
print(f'Tuned ROC Score: {roc_auc_score(y_test, test_pred)}')